In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] =  '0' #'3,2,1,0'
import sys
sys.path.append("..")
from common import *
from data   import *
from imgaug import augmenters as iaa
##----------------------------------------
from model_seresnext50_6channel import *
# from model_resnet34 import *

TkAgg
@common.py:  
	set random seed
		SEED=35202
	set cuda environment
		torch.__version__              = 0.4.1
		torch.version.cuda             = 9.0
		torch.backends.cudnn.version() = 7005
		os['CUDA_VISIBLE_DEVICES']     = 0
		torch.cuda.device_count()      = 1



In [2]:
FILE_NAME = 'baseline_25k'

In [3]:
def valid_augment(drawing, label, index):
#     image = drawing_to_image_with_color_v2(drawing, 96, 96)
    seq = iaa.Sequential([
    iaa.Crop(percent=(0.05, 0.05, \
                      0.05, 0.05), keep_size=True)
    ])
    image = drawing_to_image_with_color_aug_6channel(drawing, 96, 96, seq)
#     output_name = str(index) + '.png'
#     cv2.imwrite('../split/valid_img/' + output_name,image)
    return image, label, None


def train_augment(drawing, label, index):
    up_rand = np.random.random()
    right_rand = np.random.random()
    percent_crop = 0.1
    seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(up_rand*percent_crop, right_rand*percent_crop, \
                      (1-up_rand)*percent_crop, (1-right_rand)*percent_crop), keep_size=True)
    ])
    
    image = drawing_to_image_with_color_aug_6channel(drawing, 96, 96, seq)
#     image = drawing_to_image_with_color_v2(drawing, 96, 96)
    return image, label, None


In [4]:
### training ##############################################################

def do_valid( net, valid_loader, criterion ):

    valid_num  = 0
    probs    = []
    truths   = []
    losses   = []
    corrects = []

    for input, truth, cache in valid_loader:
        input = input.cuda()
        truth = truth.cuda()

        with torch.no_grad():
            logit   = net(input)
            prob    = F.softmax(logit,1)

            loss    = criterion(logit, truth, False)
            correct = metric(logit, truth, False)

        valid_num += len(input)
        probs.append(prob.data.cpu().numpy())
        losses.append(loss.data.cpu().numpy())
        corrects.append(correct.data.cpu().numpy())
        truths.append(truth.data.cpu().numpy())


    assert(valid_num == len(valid_loader.sampler))
    #------------------------------------------------------
    prob    = np.concatenate(probs)
    correct = np.concatenate(corrects)
    truth   = np.concatenate(truths).astype(np.int32).reshape(-1,1)
    loss    = np.concatenate(losses)


    #---
    #top = np.argsort(-predict,1)[:,:3]

    loss    = loss.mean()
    correct = correct.mean(0)

    top = [correct[0], correct[0]+correct[1], correct[0]+correct[1]+correct[2]]
    precision = correct[0]/1 + correct[1]/2 + correct[2]/3

    #----
    valid_loss = np.array([
        loss, top[0], top[2], precision
    ])

    return valid_loss

In [5]:
fold    = 0
out_dir = \
    '../../output'
initial_checkpoint = \
        '../../output/backup/887_crop.pth'

pretrain_file = None

batch_size = 256+64
epoch = 6
num_iters   = epoch * 340 * 25000 // batch_size

#     schduler  = NullScheduler(lr=0.01)
schduler = DecayScheduler(base_lr=0.01, decay=0.1, step=num_iters/3)
iter_save_interval = 2000
criterion          = softmax_cross_entropy_criterion


## setup  -----------------------------------------------------------------------------
os.makedirs(out_dir +'/checkpoint', exist_ok=True)
os.makedirs(out_dir +'/train', exist_ok=True)
os.makedirs(out_dir +'/backup', exist_ok=True)
#     backup_project_as_zip(PROJECT_PATH, out_dir +'/backup/code.train.%s.zip'%IDENTIFIER)

log = Logger()
log.open(out_dir+'/log.train_r50_6channel.txt',mode='a')
log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
log.write('\tSEED         = %u\n' % SEED)
log.write('\tPROJECT_PATH = %s\n' % PROJECT_PATH)
log.write('\t__file__     = %s\n' % FILE_NAME)
log.write('\tout_dir      = %s\n' % out_dir)
log.write('\n')
log.write('\t<additional comments>\n')
log.write('\t  ... xxx baseline  ... \n')
log.write('\n')


## dataset ----------------------------------------
log.write('** dataset setting **\n')

train_dataset = DoodleDataset('train', 'train_0', train_augment)
train_loader  = DataLoader(
                    train_dataset,
                    #sampler     = FixLengthRandomSamplerWithProbability(train_dataset, probability),
                    #sampler     = FixLengthRandomSampler(train_dataset),
                    #sampler     = ConstantSampler(train_dataset,[31]*batch_size*100),
                    sampler     = RandomSampler(train_dataset),
                    batch_size  = batch_size,
                    drop_last   = True,
                    pin_memory  = True,
                    collate_fn  = null_collate)

valid_dataset = DoodleDataset('train', 'valid_0',  valid_augment)
valid_loader  = DataLoader(
                    valid_dataset,
                    #sampler     = SequentialSampler(valid_dataset),
                    sampler     = RandomSampler(valid_dataset),
                    batch_size  = batch_size,
                    drop_last   = False,
                    pin_memory  = True,
                    collate_fn  = null_collate)


assert(len(train_dataset)>=batch_size)
log.write('batch_size = %d\n'%(batch_size))
log.write('train_dataset : \n%s\n'%(train_dataset))
log.write('valid_dataset : \n%s\n'%(valid_dataset))
log.write('\n')

## net ----------------------------------------
log.write('** net setting **\n')
net = Net().cuda()

if initial_checkpoint is not None:
    log.write('\tinitial_checkpoint = %s\n' % initial_checkpoint)
    net.load_pretrain\
    (initial_checkpoint,from_3channel=True)


--- [START 2018-11-21_01-02-30] ----------------------------------------------------------------

	SEED         = 35202
	PROJECT_PATH = C:\Users\Admin\Desktop\doodle\code
	__file__     = baseline_25k
	out_dir      = ../../output

	<additional comments>
	  ... xxx baseline  ... 

** dataset setting **
	 load df   :  339/340                   zigzag   0 min 40 sec
	 load split:  339/340                   zigzag   0 min 42 sec

	 load df   :  339/340                   zigzag   0 min 00 sec
	 load split:  339/340                   zigzag   0 min 01 sec

batch_size = 320
train_dataset : 
	split        = train_0
	mode         = train
	complexity   = simplified
	len(self.id) = 8500000


valid_dataset : 
	split        = valid_0
	mode         = train
	complexity   = simplified
	len(self.id) = 27200



** net setting **
Load imagenet pretrained SERESNEXT50
	initial_checkpoint = ../../output/backup/887_crop.pth
encoder.layer0.conv1.weight
conv1.0.weight



In [ ]:
log.write('%s\n'%(type(net)))
log.write('criterion=%s\n'%criterion)
log.write('\n')


## optimiser ----------------------------------
if 0: ##freeze
    for p in net.resnet.parameters(): p.requires_grad = False
    for p in net.encoder1.parameters(): p.requires_grad = False
    for p in net.encoder2.parameters(): p.requires_grad = False
    for p in net.encoder3.parameters(): p.requires_grad = False
    for p in net.encoder4.parameters(): p.requires_grad = False
    pass

#net.set_mode('train',is_freeze_bn=True)
#-----------------------------------------------


optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()),
                      lr=schduler.get_rate(0), momentum=0.9, weight_decay=0.0001)


iter_smooth = 20
iter_log    = 50
iter_valid  = 100
iter_save   = [0, num_iters-1]\
               + list(range(0, num_iters, iter_save_interval))#1*1000

start_iter = 0
start_epoch= 0
rate       = 0
if initial_checkpoint is not None:
#     initial_optimizer = initial_checkpoint.replace('_model.pth','_optimizer.pth')
#     checkpoint  = torch.load(initial_optimizer)
#     start_iter  = checkpoint['iter' ]
#     start_epoch = checkpoint['epoch']

    #rate = get_learning_rate(optimizer)  #load all except learning rate
    #optimizer.load_state_dict(checkpoint['optimizer'])
    #adjust_learning_rate(optimizer, rate)
    pass




log.write('schduler\n  %s\n'%(schduler))
log.write('\n')

## start training here! ##############################################
log.write('** start training here! **\n')
log.write('                    |------------ VALID -------------|-------- TRAIN/BATCH ----------|         \n')
log.write('rate   iter  epoch  | loss   acc-1  acc-3   lb       | loss   acc-1  acc-3   lb      |  time   \n')
log.write('----------------------------------------------------------------------------------------------------\n')


train_loss   = np.zeros(6,np.float32)
valid_loss   = np.zeros(6,np.float32)
batch_loss   = np.zeros(6,np.float32)
iter = 0
i    = 0
last_max_lb   = -1


start = timer()
while  iter<num_iters:
    sum_train_loss = np.zeros(6,np.float32)
    sum = 0


    optimizer.zero_grad()
    for input, truth, cache in train_loader:

        len_train_dataset = len(train_dataset)
        batch_size = len(cache)
        iter = i + start_iter
        epoch = (iter-start_iter)*batch_size/len_train_dataset + start_epoch
        num_samples = epoch*len_train_dataset


        if (iter % iter_valid==0) and (iter!=0):
            net.set_mode('valid')
            valid_loss = do_valid(net, valid_loader, criterion)
            net.set_mode('train')

            ##--------
            # lb    = valid_loss[7]
            # loss  = valid_loss[0] + valid_loss[4]
            # last_max_lb = max(last_max_lb,lb)
            # if last_max_lb-lb<0.005:
            #     iter_save += [iter,]
            # if loss-last_min_loss<0.005:
            #     iter_save += [iter,]

            asterisk = '*' if iter in iter_save else ' '
            ##--------

            print('\r',end='',flush=True)
            log.write('%0.4f %5.1f %6.1f | %0.3f  %0.3f  %0.3f  (%0.3f)%s  | %0.3f  %0.3f  %0.3f  (%0.3f)  | %s' % (\
                     rate, iter/1000, epoch,
                     valid_loss[0], valid_loss[1], valid_loss[2], valid_loss[3],asterisk,
                     train_loss[0], train_loss[1], train_loss[2], train_loss[3],
                     time_to_str((timer() - start),'min'))
            )
            log.write('\n')
            time.sleep(0.01)

        #if 0:
        if iter in iter_save:
            torch.save(net.state_dict(),out_dir +'/checkpoint/%08d_model.pth'%(iter))
            torch.save({
                #'optimizer': optimizer.state_dict(),
                'iter'     : iter,
                'epoch'    : epoch,
            }, out_dir +'/checkpoint/%08d_optimizer.pth'%(iter))
            pass




        # learning rate schduler -------------
        lr = schduler.get_rate(iter)
        if lr<0 : break
        adjust_learning_rate(optimizer, lr)
        rate = get_learning_rate(optimizer)



        # one iteration update  -------------
        #net.set_mode('train',is_freeze_bn=True)
        net.set_mode('train')
        input = input.cuda()
        truth = truth.cuda()

        logit = data_parallel(net, input)
        loss  = criterion(logit, truth)
        precision, top = metric(logit, truth)


        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        #torch.nn.utils.clip_grad_norm(net.parameters(), 1)


        # print statistics  ------------
        batch_loss[:4] = np.array(( loss.item(), top[0].item(), top[2].item(), precision.item(),))
        sum_train_loss += batch_loss
        sum += 1
        if iter%iter_smooth == 0:
            train_loss = sum_train_loss/sum
            sum_train_loss = np.zeros(6,np.float32)
            sum = 0


        print('\r',end='',flush=True)
        print('%0.4f %5.1f %6.1f | %0.3f  %0.3f  %0.3f  (%0.3f)%s  | %0.3f  %0.3f  %0.3f  (%0.3f)  | %s' % (\
                     rate, iter/1000, epoch,
                     valid_loss[0], valid_loss[1], valid_loss[2], valid_loss[3],' ',
                     batch_loss[0], batch_loss[1], batch_loss[2], batch_loss[3],
                     time_to_str((timer() - start),'min'))
        , end='',flush=True)
        i=i+1



    pass  #-- end of one data loader --
pass #-- end of all iterations --


if 1: #save last
    torch.save(net.state_dict(),out_dir +'/checkpoint/%d_model.pth'%(i))
    torch.save({
        'optimizer': optimizer.state_dict(),
        'iter'     : i,
        'epoch'    : epoch,
    }, out_dir +'/checkpoint/%d_optimizer.pth'%(i))

log.write('\n')



<class 'model_seresnext50_6channel.Net'>
criterion=<function softmax_cross_entropy_criterion at 0x0000026014779158>

schduler
  (Exp) Decay Learning Rates
base_lr=0.010, decay=0.100, step=53125.000

** start training here! **
                    |------------ VALID -------------|-------- TRAIN/BATCH ----------|         
rate   iter  epoch  | loss   acc-1  acc-3   lb       | loss   acc-1  acc-3   lb      |  time   
----------------------------------------------------------------------------------------------------


..\data.py:260: RuntimeWarning: invalid value encountered in true_divide
  colors = plt.cm.jet(np.arange(0,P_part1+1)/(P_part1))
D:\Anaconda\lib\site-packages\matplotlib\colors.py:496: RuntimeWarning: invalid value encountered in less
  cbook._putmask(xa, xa < 0.0, -1)
D:\Anaconda\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


0.0100   0.1    0.0 | 0.000  0.000  0.000  (0.000)   | 0.800  0.800  0.922  (0.857)  |  0 hr 01 min

D:\Anaconda\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


0.0100   0.1    0.0 | 0.793  0.797  0.924  (0.855)   | 0.870  0.785  0.914  (0.844)  |  0 hr 02 min
0.0100   0.2    0.0 | 0.767  0.803  0.926  (0.860)   | 0.773  0.804  0.927  (0.861)  |  0 hr 05 min
0.0100   0.3    0.0 | 0.758  0.803  0.930  (0.861)   | 0.724  0.813  0.933  (0.868)  |  0 hr 07 min
0.0100   0.4    0.0 | 0.758  0.805  0.928  (0.862)   | 0.773  0.805  0.927  (0.861)  |  0 hr 10 min
0.0100   0.5    0.0 | 0.762  0.803  0.928  (0.861)   | 0.725  0.809  0.933  (0.866)  |  0 hr 13 min
0.0100   0.6    0.0 | 0.754  0.807  0.929  (0.863)   | 0.792  0.801  0.923  (0.857)  |  0 hr 15 min
0.0100   0.7    0.0 | 0.751  0.807  0.929  (0.863)   | 0.743  0.811  0.929  (0.866)  |  0 hr 18 min
0.0100   0.8    0.0 | 0.753  0.806  0.929  (0.863)   | 0.752  0.813  0.926  (0.865)  |  0 hr 20 min
0.0100   0.9    0.0 | 0.748  0.806  0.929  (0.863)   | 0.791  0.802  0.925  (0.859)  |  0 hr 23 min
0.0100   1.0    0.0 | 0.748  0.808  0.930  (0.864)   | 0.813  0.792  0.918  (0.850)  |  0 hr 25 min


0.0100   8.2    0.3 | 0.723  0.813  0.932  (0.868)   | 0.787  0.801  0.926  (0.858)  |  3 hr 31 min
0.0100   8.3    0.3 | 0.722  0.813  0.932  (0.868)   | 0.763  0.802  0.926  (0.859)  |  3 hr 33 min
0.0100   8.4    0.3 | 0.723  0.811  0.931  (0.866)   | 0.769  0.811  0.928  (0.865)  |  3 hr 36 min
0.0100   8.5    0.3 | 0.714  0.815  0.932  (0.869)   | 0.768  0.806  0.930  (0.863)  |  3 hr 39 min
0.0100   8.6    0.3 | 0.720  0.814  0.933  (0.869)   | 0.747  0.808  0.929  (0.863)  |  3 hr 41 min
0.0100   8.7    0.3 | 0.723  0.813  0.932  (0.868)   | 0.775  0.803  0.929  (0.861)  |  3 hr 44 min
0.0100   8.8    0.3 | 0.723  0.811  0.932  (0.867)   | 0.763  0.808  0.928  (0.864)  |  3 hr 46 min
0.0100   8.9    0.3 | 0.724  0.811  0.932  (0.867)   | 0.798  0.804  0.922  (0.858)  |  3 hr 49 min
0.0100   9.0    0.3 | 0.719  0.814  0.933  (0.869)   | 0.771  0.806  0.924  (0.861)  |  3 hr 51 min
0.0100   9.1    0.3 | 0.719  0.811  0.932  (0.867)   | 0.776  0.801  0.923  (0.857)  |  3 hr 54 min


0.0100  16.3    0.6 | 0.710  0.815  0.933  (0.870)   | 0.755  0.814  0.926  (0.866)  |  6 hr 59 min
0.0100  16.4    0.6 | 0.714  0.816  0.933  (0.871)   | 0.761  0.806  0.926  (0.862)  |  7 hr 02 min
0.0100  16.5    0.6 | 0.719  0.815  0.932  (0.870)   | 0.767  0.806  0.930  (0.863)  |  7 hr 04 min
0.0100  16.6    0.6 | 0.712  0.813  0.934  (0.869)   | 0.766  0.807  0.930  (0.864)  |  7 hr 07 min
0.0100  16.7    0.6 | 0.711  0.813  0.933  (0.869)   | 0.801  0.804  0.924  (0.858)  |  7 hr 10 min
0.0100  16.8    0.6 | 0.712  0.814  0.933  (0.869)   | 0.732  0.811  0.932  (0.866)  |  7 hr 12 min
0.0100  16.9    0.6 | 0.712  0.814  0.933  (0.869)   | 0.589  0.847  0.944  (0.892)  |  7 hr 14 min

In [ ]:
input.shape